In [3]:
import urllib3
from copy import deepcopy
import json
from multiprocessing.pool import ThreadPool
import re

from bs4 import BeautifulSoup

## Importamos o metodo de limpeza de datos

In [4]:
#Librarías para limpieza de datos
import string
import nltk
from nltk.corpus import stopwords

def limparcodigo(texto):
    clean_texto=''
    punctuation=[]
    for s in string.punctuation:
        punctuation.append(str(s))
    sp_punctuation = ["¿", "¡", "“", "”", "…", ":", "–", "»", "«",'->','(',')']

    punctuation += sp_punctuation

    punctuation[:10] 
    stop_words = stopwords.words('spanish') 

    stop_words += ["\u200b", "\xa0",'a','ante','bajo','cabe','con','contra','de','desde','en','entre','hacia','hasta','para','por','segun','sin','so','sobre','tras','mediante','durante','excepto','salvo','el','de','y','es','se','en','con',"para", "como", "puede","cómo", "hacer", "forma", "parte", "hace", "además", "según", "pueden", "ser",'chegar','encuadren','luego','fai','fin','dos','poden','cualquier','tratan','encontrar','chámase','mayor','dos','poden','non','seu','base','poder','luego','cales','destas','mismos','característica','tan','después','llegar','sacar','mediante','é','punto','bien','unha'] #Añadimos algunos caractéres que hemos encontrado

    stop_words[:10]

    for sw in stop_words:
        clean_texto = texto.lower().replace(sw,"")

    for p in punctuation:
        clean_texto = clean_texto.replace(p,"")
 
    for stop in stop_words:
        clean_texto_list = clean_texto.split()
        clean_texto_list = [i.strip() for i in clean_texto_list]
        try:
            while stop in clean_texto_list: clean_texto_list.remove(stop)
        except:
            print("Error")
            pass
        clean_texto= " ".join(clean_texto_list)

    clean_texto
    lista_texto = clean_texto.split(" ")

    palabras = []
    #Paso intermedio para eliminar palabras muy cortas (emoticonos,etc) y muy largas (ulr o similar) que se nos hayan pasado:

    for palabra in lista_texto:
        if (len(palabra)>=3 and len(palabra)<18):
            palabras.append(palabra)
    resultado=' '.join(palabras)
    return resultado


In [5]:
def conseguirLink(link):
    ua = "Mozilla/5.0 (Linux; U; Android 2.2; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1"  
    h = {"User-Agent": ua}
    httpPool = urllib3.PoolManager()
    return httpPool.request('GET',link,fields=None,headers=h)

In [7]:

claves = {}

def get_data(map):
    dato=map[1]
    datos=[]
    soup = BeautifulSoup(conseguirLink(dato[1]).data,'html.parser')
    urls=[]
    for text in soup.find_all(dato[2],{"class":dato[3]} if len(dato[3])>0 else None):
        texto = text.text.replace("\n","")
        texto = texto.replace("\r","")
        texto = texto.replace("\t","")
        if dato[4] == ".":
            enlace = text
            bEnlace = text[dato[5]]
        else:
            enlace=text.findChildren(dato[4] , recursive=False)
            if len(enlace)==0:
                continue
            bEnlace = enlace[0][dato[5]]
        if bEnlace.startswith("/"):
            bEnlace=dato[1]+bEnlace
        datosenlace=[texto,bEnlace,0]
        datos.append(datosenlace)
        urls.append(bEnlace)
    claves[dato[0]]=urls
    return datos

In [15]:
URLS = {"La Voz de Galicia": {"url": "https://www.lavozdegalicia.es", "tag": "h4", "clase": "a-min-headline"},
"El Mundo": {"url": "https://www.elmundo.es", "tag": "h2", "clase": "ue-c-cover-content-link"}}


In [16]:
allTitulares = deque()
enderezos=json.loads(json.dumps(URLS))
for auxURL in enderezos:
    allTitulares.append(dameTitulares(enderezos[auxURL]['url'], enderezos[auxURL]['tag'], enderezos[auxURL]['clase']))
for auxTitular in allTitulares:
    print(auxTitular)


Filtrando resultados do medio: https://www.lavozdegalicia.es
Filtrando resultados do medio: https://www.elmundo.es
deque([['El PSOE y sus socios blindan a Tsunami y a los CDR al incluir el terrorismo en la ley de amnistía', '/noticia/espana/2024/01/23/junts-asegura-tener-acuerdo-psoe-retocar-referencia-terrorismo-ley-amnistia/00031706007975721867380.htm'], ['Los protagonistas del intento de amaño del Badajoz-Pontevedra: «Hablamos con el míster y no jugamos»', '/noticia/pontevedra/2024/01/23/protagonistas-intento-amano-badajoz-pontevedra-hablamos-mister-preferimos-jugar-partido/00031706008945381481123.htm'], ['En directo: Celta 0 - 1 Real Sociedad', '/retransmision/gradario/2024/01/23/directo-celta-real-sociedad/01071706038808428450532.htm'], ['Los promotores de la mayor granja de cerdo celta: «En Guijuelo dicen que estamos locos»', '/noticia/somosagro/ganaderia/2024/01/23/empresa-sarria-abrira-incio-gran-granja-cerdo-celta/0003_202401M23C1991.htm'], ['La participación en las ciudades s